## Je dors pendant que mon agent fait le taff!!

## Installation des packages

In [1]:
!pip install yfinance

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
!pip install pandas numpy torch

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


## Importation des bibliothèques

In [3]:
import yfinance as yf           # Pour télécharger les données financières de Yahoo Finance
import pandas as pd             # Pour la manipulation des données tabulaires
import numpy as np              # Pour les calculs numériques
import torch                    # Framework d'apprentissage profond
import torch.nn as nn           # Modules de réseaux de neurones
import torch.optim as optim     # Optimiseurs pour l'entraînement des modèles
import random                   # Pour la génération de nombres aléatoires
from collections import deque   # Structure de données pour stocker une séquence limitée

# Définition du symbole boursier et de la période
symbol = "YMA.F"               # Ticker de Yamaha .
start_date = "2020-01-01"     # Date de début de l'historique
end_date = "2025-03-12"       # Date de fin de l'historique

#téléchargement des données
data = yf.download(symbol, start=start_date, end=end_date)

YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed


- Yahoo Finance API (yfinance) : Interface pour acceder gratuitement aux données boursières historiques

- Pandas et NumPy : Outils essentiels pour l'analyse de données

- PyTorch : Framework d'apprentissage profond populaire, particulièrement adapté pour les réseaux de neurones.

- Struxture d'apprentissage par renforcement : L'importation "deque" suggère la mise en place possible d'un algorithme d'apprentissage par renforcement comme le deep Q learining qui utilise souvent le "replay memory" implémentée avec "deque"



## Mon analyse technique

In [4]:
data['SMA_5'] = data['Close'].rolling(window=5).mean()
data['SMA_20'] = data['Close'].rolling(window=20).mean()
data['Returns'] = data['Close'].pct_change()

- La première ligne crée une colonne qui contient la moyenne mobile simple des derniers jours du prix de cloture

- La deuxième calcul la moyenne mobile longue sur les 20 derniers jours. Ici c'est beaucoup plus pour les tendances fondamentales

- La dernière ligne calcule les rendements quotidiens en pourcentage. La fonction "pct_change()" calcule le changement en pourcentage d'une période à l'autre 


## Suppression des valeurs manquantes et Réinitialisation des indices

In [5]:
data.dropna(inplace=True)
data.reset_index(drop=True, inplace=True)

- Les premières valeurs des colonnes SMA_20 et 5 seront NaN car il n'y pas assez de données pour calculer la moyenne sur les premières périodes. Par exemple SMA_5 aura des NaN pour les 4 premiers jours

- Après avoir supprimé les lignes avec "dropna", les indices originaux sont conservés, ce qui crée des trous dans la sequence des indices. La dernière ligne réinitialise qu'oms forment une séquence continue de 0 à n-1.

## Les actions possibles à prendre sur le marché

In [6]:
ACTIONS = {0: "HOLD", 1: "BUY", 2: "SELL"}

## Extraction des données avec ma fonction get_state

In [7]:
def get_state(data, index):
    return np.array([
        float(data.loc[index, 'Close']),
        float(data.loc[index, 'SMA_5']),
        float(data.loc[index, 'SMA_20']),
        float(data.loc[index, 'Returns'])
    ])

Cette fonction extrait les information pertinentes à un moment précis( indexé par "index") du DataFrame "data" et les renvoie sous forme d'un tableau numpy, qui servira d"etat" ou de "feature vector" pour mon modèle

La cellule suivante vérifie juste si toutes les colonnes sont bien présentes

In [8]:
data.columns

MultiIndex([(  'Close', 'YMA.F'),
            (   'High', 'YMA.F'),
            (    'Low', 'YMA.F'),
            (   'Open', 'YMA.F'),
            ( 'Volume', 'YMA.F'),
            (  'SMA_5',      ''),
            ( 'SMA_20',      ''),
            ('Returns',      '')],
           names=['Price', 'Ticker'])

## Simuler l'environnement de trading dans lequel notre Agent pour agir 

In [9]:
class TradingEnvironment:
    def __init__(self, data):
        self.data = data
        self.initial_balance = 10000
        self.balance = self.initial_balance
        self.holdings = 0
        self.index = 0

    def reset(self):
        self.balance = self.initial_balance
        self.holdings = 0
        self.index = 0
        return get_state(self.data, self.index)
    
    def step(self, action):
        price = float(self.data.loc[self.index, 'Close'])
        reward = 0
        if action == 1 and self.balance >= price : #buy
            self.holdings = self.balance // price 
            self.balance -= self.holdings * price
        elif action == 2 and self.holdings > 0:
            self.balance += self.holdings * price
            self.holdings = 0
        self.index += 1
        done = self.index >= len(self.data) - 1
        if done:
            reward =self.balance - self.initial_balance
        next_state = get_state(self.data, self.index) if not done else None
        return next_state, reward, done, {}

## On définie l'architecture de notre réseau de neurones pour l'apprentissage par renforcement, spéciquement la méthode du DQN

In [10]:
class DQN(nn.Module):
    def __init__(self, state_size, action_size):
        super(DQN, self).__init__()
        self.fc1 = nn.Linear(state_size, 64)
        self.fc2 = nn.Linear(64, 64)
        self.fc3 = nn.Linear(64, action_size)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        return self.fc3(x)

Dans l'algorithme DQN :

On estime la "valeur Q" de chaque action possible dans un etat donné

La valeur Q est la récompense totale espérée en prenant cette action puis en suivant la polotique optimale

L'agent choisira la valeur avec la plus grande valeur Q

## DQN Agent

In [11]:
class DQNAgent:
    def __init__(self, state_size, action_size):
        self.state_size = state_size
        self.action_size = action_size
        self.memory = deque(maxlen=2000)
        self.gamma = 0.95  # Discount factor
        self.epsilon = 1.0  # Exploration rate
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.995
        self.learning_rate = 0.001
        self.model = DQN(state_size, action_size)
        self.optimizer = optim.Adam(self.model.parameters(), lr=self.learning_rate)
        self.criterion = nn.MSELoss()

    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))

    def act(self, state):
        if random.uniform(0, 1) < self.epsilon:
            return random.choice(list(ACTIONS.keys()))
        state = torch.FloatTensor(state).unsqueeze(0)
        with torch.no_grad():
            q_values = self.model(state)
        return torch.argmax(q_values).item()
    
    def replay(self, batch_size):
        if len(self.memory) < batch_size:
            return
        minibatch = random.sample(self.memory, batch_size)
        for state, action, reward, next_state, done in minibatch:
            target = reward
            if not done:
                next_state_tensor = torch.FloatTensor(next_state).unsqueeze(0)
                target += self.gamma * torch.max(self.model(next_state_tensor)).item()
            state_tensor = torch.FloatTensor(state).unsqueeze(0)
            target_tensor = self.model(state_tensor).clone().detach()
            target_tensor[0][action] = target
            self.optimizer.zero_grad()
            output = self.model(state_tensor)
            loss = self.criterion(output, target_tensor)
            loss.backward()
            self.optimizer.step()
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay


- Experience Replay ; Stocke et réutilise les expériences passées pour améliorer la stabilité de l'apprentisssage 

- Politique Epsilon-Greedy : Balance entre l'exploration (actions aléatoires) et exploitation (actions optimales selon le modèle)

- Apprentissage hors politique : Apprend la politique optimale independamment de la politique d'exploration

- Approximation de fontion Q : Utilise un réseau de neurones pour approximer la fonction de valeur Q

## Boucle d'entrainement 

In [12]:
env = TradingEnvironment(data)
agent = DQNAgent(state_size=4, action_size=3)
batch_size = 32
episodes = 500
total_rewards = []

for episode in range(episodes):
    state = env.reset()
    done = False
    total_reward = 0

    while not done:
        action = agent.act(state)
        next_state, reward, done, _ = env.step(action)
        agent.remember(state, action, reward, next_state, done)
        state = next_state
        total_reward += reward

    agent.replay(batch_size)
    total_rewards.append(total_reward)
    print(f"Episode {episode+1}/{episodes}, Total Reward: {total_reward}")

print("Training Complete!")

C:\Users\nguen\AppData\Local\Temp\ipykernel_26344\3443828189.py:3: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  float(data.loc[index, 'Close']),
C:\Users\nguen\AppData\Local\Temp\ipykernel_26344\3443828189.py:4: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  float(data.loc[index, 'SMA_5']),
C:\Users\nguen\AppData\Local\Temp\ipykernel_26344\3443828189.py:5: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  float(data.loc[index, 'SMA_20']),
C:\Users\nguen\AppData\Local\Temp\ipykernel_26344\3443828189.py:6: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  float(data.loc[index, 'Returns'])
C:\Users\nguen\App

Episode 1/500, Total Reward: -11334.953857421875
Episode 2/500, Total Reward: -10499.667358398438
Episode 3/500, Total Reward: -10181.473876953125
Episode 4/500, Total Reward: -10070.225830078125
Episode 5/500, Total Reward: -11192.478271484375
Episode 6/500, Total Reward: -10718.3505859375
Episode 7/500, Total Reward: -10327.33642578125
Episode 8/500, Total Reward: -10931.122436523438
Episode 9/500, Total Reward: -10266.432006835938
Episode 10/500, Total Reward: -11094.059936523438
Episode 11/500, Total Reward: -10370.241455078125
Episode 12/500, Total Reward: -10320.640625
Episode 13/500, Total Reward: -11016.754150390625
Episode 14/500, Total Reward: -10429.098388671875
Episode 15/500, Total Reward: -10290.146484375
Episode 16/500, Total Reward: -11193.811279296875
Episode 17/500, Total Reward: -10878.080688476562
Episode 18/500, Total Reward: -11183.6181640625
Episode 19/500, Total Reward: -11303.805053710938
Episode 20/500, Total Reward: -11224.236694335938
Episode 21/500, Total R

Analyse du processus d'apprentissage

Ce que fait réellement cette boucle d'entraînement :

- Exploration vs Exploitation : Au début, l'agent effectue principalement des choix aléatoires (exploration, epsilon ≈ 1). Au fur et à mesure que l'entraînement progresse, l'agent s'appuie davantage sur son modèle appris (exploitation, epsilon diminue).

- Apprentissage incrémental : À chaque épisode, l'agent affine son modèle en se basant sur les expériences passées.

- Retour d'information différé : La récompense n'est obtenue qu'à la fin de l'épisode, ce qui représente un défi pour l'apprentissage (problème d'attribution de crédit).

- Suivi des performances : L'affichage des récompenses totales permet de suivre l'amélioration de l'agent au fil du temps.


## TESTING

In [13]:
test_env = TradingEnvironment(data)
state = test_env.reset()
done = False
# simulate a trading session using the trained agent
while not done:
    # always choose the best action (exploitation)
    action = agent.act(state)
    next_state, reward, done, _ = test_env.step(action)
    state = next_state if next_state is not None else state
final_balance = test_env.balance
profit = final_balance - test_env.initial_balance
print(f"Final Balance after testing: ${final_balance:.2f}")
print(f"Total Profit: ${profit:.2f}")

C:\Users\nguen\AppData\Local\Temp\ipykernel_26344\3443828189.py:3: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  float(data.loc[index, 'Close']),
C:\Users\nguen\AppData\Local\Temp\ipykernel_26344\3443828189.py:4: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  float(data.loc[index, 'SMA_5']),
C:\Users\nguen\AppData\Local\Temp\ipykernel_26344\3443828189.py:5: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  float(data.loc[index, 'SMA_20']),
C:\Users\nguen\AppData\Local\Temp\ipykernel_26344\3443828189.py:6: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  float(data.loc[index, 'Returns'])
C:\Users\nguen\App

Final Balance after testing: $-1040.91
Total Profit: $-11040.91


Ce code teste mon agent de trading après l'entraînement. Il simule une session de trading complète en suivant strictement la politique apprise (exploitation pure, sans exploration). Pour chaque état, l'agent choisit la meilleure action selon son modèle actuel, puis l'environnement évolue en conséquence.

À la fin, le code affiche le solde final et le profit total réalisé par rapport au capital initial. Cette évaluation est importante pour mesurer la performance réelle de l'agent dans un environnement de trading.

## MERCI